In [1]:
import urllib.request
import datetime
import json

def fetch(url):
   req = urllib.request.Request(url)
   with urllib.request.urlopen(req) as response:
      return response.read()
   
def save_data(data, name):
   playlists_json = json.loads(data.decode('utf-8'))
   with open(name, "w") as f:
      json.dump(playlists_json, f)

def get_playlists_url(user_id, client_id):
   limit = 1000
   return f"https://api-v2.soundcloud.com/users/{user_id}/playlists?limit={limit}&client_id={client_id}"

def get_tracks_url(user_id, client_id):
   limit = 10_000
   return f"https://api-v2.soundcloud.com/users/{user_id}/likes?limit={limit}&client_id=${client_id}"

def get_info(ctx):
   return {
      "url": f"",
      "name": f"",
   }

def load_env():
    import tomllib
    with open("../env.toml", "rb") as f:
        env_data = tomllib.load(f)
    return env_data["soundcloud"]
env_data = load_env()

def get_pages():
   user_id = env_data["user_id"]
   client_id = env_data["client_id"]
   today = datetime.date.today()
   return {
      f"{get_playlists_url(user_id, client_id)}": f"playlists-{today}.json",
      f"{get_tracks_url(user_id, client_id)}": f"tracks-{today}.json",
   }

# [
#    save_data(fetch(page_url), page_name)
#    for (page_url, page_name) in get_pages()
# ]

In [ ]:
import polars as pl

df_playlist = (
pl.DataFrame(df_playlist_json["collection"])
    .explode("collection")
    .unnest("collection")
    .explode("tracks")
    .select([
        "title",
        pl.col("tracks").struct.field("id").alias("track.id")
    ])
)

df_playlist

df_likes = (
pl.DataFrame(df_likes_json["collection"])
    .explode("collection")
    .unnest("collection")
    .select(
        pl.col("*").exclude("kind", "created_at")
    )
    .unnest("track")
    .select([
        "id",
        pl.col("title").str.to_lowercase(),
        "permalink_url",
        "playback_count",
        "created_at",
        # "user",
        pl.col("user").struct.field("id").alias("user.id"),
        pl.col("user").struct.field("username").str.to_lowercase().alias("user.username"),
    ])
)

df_likes_full = df_likes.join(df_playlist, left_on="id", right_on="track.id", how="left", suffix="_playlist")
df_likes_full

In [ ]:
df_usernames = (
    df_likes_full
        .select(
            pl.col("user.id").alias("userid"),
            pl.col("user.username").alias("username"),
        )
        .unique()
)
df_usernames

In [ ]:
df_playlist_full = (
    df_playlist
        .join(df_likes, left_on="track.id", right_on="id", how="inner")
        .select(
            pl.col("*").exclude("user.username", "title_right"),
            pl.col("title_right").str.to_lowercase().alias("track.title"),
            pl.col("user.username").str.to_lowercase()
        )
)
df_playlist_full


In [ ]:
df_playlist_user_count = (
    df_playlist_full
        .join(df_usernames, how="cross")
        .filter(
            (pl.col("user.id") == pl.col("userid")) |
            (pl.col("track.title").str.contains(pl.col("username"), literal=True)),
        )
        .select(
            "title",
            "username",
            pl.col("username").count().over("username").alias("username_count"),
            pl.col("username").count().over("username", "title").alias("username_count_all"),
        )
        .unique(["title", "username"])
        .sort(["username_count", "username"], descending=True)
)
df_playlist_user_count

In [ ]:
df_likes_user_count = (
    df_likes_full
        .join(
            df_usernames
            , how="cross"
        )
        .filter(
            (pl.col("user.id") == pl.col("userid")) |
            (pl.col("title").str.contains(pl.col("username"), literal=True)),
        )
        .with_columns(
            pl
                .when(
                    pl.col("user.id") != pl.col("userid")
                )
                .then(pl.col("username"))
                .otherwise(pl.col("user.username"))
                .alias("username_join"),
        )
        .select(
            "username_join",
            pl.col("username_join").count().over("username_join").alias("username_count"),
        )
        .unique("username_join")
        .sort("username_count", descending=True)
)
df_likes_user_count

In [ ]:
def tracks_of_author(author):
    return (
        df_likes_full
            .filter(
                pl.col("user.username").str.contains(author, literal=True) |
                pl.col("title").str.contains(author, literal=True)
            )
            .select(
                "*", 
                pl.col("id").count().alias("count"),
            )
    )

# df = tracks_of_author("")
# df